In [ ]:
import pandas as pd
import cv2
import numpy as np

# Noms des fichiers
data_file = 'données POR.xlsx'
video_file = 'Test Your Awareness.avi'
output_file = 'video_with_por_output.avi'

# 1. Chargement des données de points de regard
def load_por_data(file_path):
    """
    Charge les données de points de regard (POR) depuis un fichier Excel.
    Normalise la colonne 'Time' pour démarrer à 0 et utilise les coordonnées de l'œil droit pour ce test.

    - Elle reçoit : Le chemin vers le fichier Excel.
    - Elle fait : Convertit la colonne 'Time' pour un démarrage à 0, par ce que je n'ai pas compris à quoi correspond le time (timestamp d'après chatgpt, surement est pris à partir d'une réference que je n'ai pas).
    - Elle renvoie : Un DataFrame avec 'time', 'R POR X [px]', et 'R POR Y [px]'.
    """
    # pd.read_excel(file_path) : Charge le fichier Excel et renvoie un DataFrame avec les données.
    # df[['Time', 'R POR X [px]', 'R POR Y [px]']] : Sélectionne les colonnes nécessaires du DataFrame.
    # rename(...) : Renomme les colonnes pour faciliter la manipulation.
    # df['time'] - df['time'].iloc[0] : Normalise 'time' pour démarrer à 0.
    df = pd.read_excel(file_path)
    df = df[['Time', 'R POR X [px]', 'R POR Y [px]']].rename(columns={'Time': 'time', 'R POR X [px]': 'x', 'R POR Y [px]': 'y'})
    df['time'] = df['time'] - df['time'].iloc[0]  # Normaliser le temps pour démarrer à 0
    return df

# 2. Mise à l'échelle des coordonnées par ce qu'ils sont trop grand pour être en pixel
def scale_coordinates(por_data, frame_width, frame_height):
    """
    Met à l'échelle les coordonnées de points de regard pour les adapter à la résolution de la vidéo.
    Utilise les valeurs maximales trouvées pour offrir un champ de mouvement maximal.

    - Elle reçoit : Le DataFrame de points de regard et les dimensions de la vidéo.
    - Elle fait : Normalise 'x' et 'y' en fonction des valeurs maximales observées et arrondit vers le haut.
    - Elle renvoie : Un DataFrame avec des coordonnées mises à l'échelle.
    """
    # max_x, max_y : Valeurs maximales définies pour les axes X et Y, permettant une mise à l'échelle optimale.
    max_x, max_y = 18202624, 11357281  # Valeurs maximales pour R POR X et Y j'ai lu le fichier en python et ai recupéré la plus grande valeur de chacun des coordonnées
    # np.ceil(...) : Applique un arrondi à l'excès après la normalisation pour éviter la perte de mouvement.
    por_data['x'] = np.ceil((por_data['x'] / max_x) * frame_width)
    por_data['y'] = np.ceil((por_data['y'] / max_y) * frame_height)
    return por_data

# 3. Incrustation du point de regard dans chaque frame
def overlay_gaze_point(video_file, por_data, output_file):
    """
    Parcourt chaque frame de la vidéo et y incruste le point de regard.

    - Elle reçoit : Le fichier vidéo, le DataFrame de points de regard, et le nom du fichier de sortie.
    - Elle fait : Incruste un cercle rouge dans chaque frame selon les coordonnées (x, y) de l'œil.
    - Elle renvoie : Sauvegarde une nouvelle vidéo avec le point de regard dans chaque frame.
    """
    # cv2.VideoCapture(video_file) : Ouvre le fichier vidéo pour en lire les frames.
    cap = cv2.VideoCapture(video_file)
    # cap.get(cv2.CAP_PROP_FRAME_WIDTH) et cap.get(cv2.CAP_PROP_FRAME_HEIGHT) : Récupèrent la largeur et la hauteur de la vidéo.
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    # fps : Nombre d'images par seconde basé sur le nombre de lignes de por_data et de frames de la vidéo.
    fps = len(por_data) / cap.get(cv2.CAP_PROP_FRAME_COUNT)
    # cv2.VideoWriter : Initialise la vidéo de sortie avec les dimensions et fps définis.
    out = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height))

    # Dernières coordonnées valides pour gérer les valeurs manquantes
    last_x, last_y = None, None

    # Itération sur chaque frame et chaque ligne de données
    for index, row in por_data.iterrows():
        # cap.read() : Lit une nouvelle frame de la vidéo. Renvoie un booléen 'ret' pour indiquer si la lecture est réussie, et la frame elle-même.
        ret, frame = cap.read()
        if not ret:
            print("Vidéo terminée ou problème lors de la lecture de la frame.")
            break

        # Récupérer les coordonnées ou utiliser les dernières valides si manquantes
        x, y = int(row['x']), int(row['y'])
        # pd.isna(...) : Vérifie si les coordonnées sont NaN (valeurs manquantes)
        if pd.isna(x) or pd.isna(y):
            x, y = last_x, last_y
        else:
            last_x, last_y = x, y

        # cv2.circle(frame, (x, y), 10, (0, 0, 255), -1) : Incruste un cercle rouge de rayon 10 pixels aux coordonnées (x, y) sur la frame.
        cv2.circle(frame, (x, y), 10, (0, 0, 255), -1)

        # out.write(frame) : Écrit la frame modifiée dans le fichier vidéo de sortie.
        out.write(frame)

        # cv2.imshow(...) : Affiche la frame en cours de traitement dans une fenêtre appelée 'POR Overlay'.
        cv2.imshow('POR Overlay', frame)
        # cv2.waitKey(1) & 0xFF == ord('q') : Attend une touche pressée. Si 'q' est pressé, arrête le processus.
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Arrêt manuel par l'utilisateur.")
            break

    # cap.release() et out.release() : Libèrent les ressources vidéo pour le fichier d'entrée et de sortie.
    cap.release()
    out.release()
    # cv2.destroyAllWindows() : Ferme toutes les fenêtres d'affichage créées par OpenCV.
    cv2.destroyAllWindows()
    print(f"La vidéo avec POR a été sauvegardée localement sous le nom : {output_file}")

# Exécution principale
por_data = load_por_data(data_file)
cap = cv2.VideoCapture(video_file)
frame_width, frame_height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
por_data = scale_coordinates(por_data, frame_width, frame_height)
overlay_gaze_point(video_file, por_data, output_file)
cap.release()


Vidéo terminée ou problème lors de la lecture de la frame.
La vidéo avec POR a été sauvegardée localement sous le nom : video_with_por_output.avi
